In [11]:
import os
import sys
import numpy as np
import pandas as pd
import torch
# from pyod.models.iforest import IForest
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import auc

In [15]:
def find_best_percent(result, granularity_all=1000):
    """
    find threshold
    :param result: sorted result
    :param granularity_all: granularity_all
    """
    max_percent = 0
    best_n = 1
    print("threshold tuning start:")
    for n in tqdm(range(1, 100)):
        head_n = n / granularity_all
        data_length = max(round(len(result) * head_n), 1)
        count_dist = count_entries(result.loc[:data_length - 1], 'label')
        try:
            percent = count_dist['1'] / (count_dist['0'] + count_dist['1'])
        except KeyError:
            print("can't find n%,take 1%")
            percent = 0.01
        if percent > max_percent:
            max_percent = percent
            best_n = n
    print("top %d / %s is the highest, %s" % (granularity_all, best_n, max_percent))
    return best_n, max_percent, granularity_all

def count_entries(df, col_name):
    """
    count
    """
    count_dist = {'0': 0, '1': 0}
    col = df[col_name]
    for entry in col:
        if str(int(entry)) in count_dist.keys():
            count_dist[str(int(entry))] = count_dist[str(int(entry))] + 1
        else:
            count_dist[str(int(entry))] = 1
    return count_dist

def find_best_result(threshold_n, result, dataframe_std):
    """
    find_best_result
    :param threshold_n: threshold
    :param result: sorted result
    :param dataframe_std: label
    """
    best_result, best_h, best_re, best_fa, best_f1, best_precision = None, 0, 0, 0, 0, 0
    best_auroc = 0
    for h in tqdm(range(50, 1000, 50)):
        train_result = charge_to_car(threshold_n, result, head_n=h)
        f1, recall, false_rate, precision, accuracy, auroc = evaluation(dataframe_std, train_result)
        if auroc >= best_auroc:
            best_f1 = f1
            best_h = h
            best_re = recall
            best_fa = false_rate
            best_result = train_result
            best_auroc = auroc
    return best_result, best_h, best_re, best_fa, best_f1, best_auroc

def charge_to_car(threshold_n, rec_result, head_n=92):
    """
    mapping from charge to car
    :param threshold_n: threshold
    :param rec_result: sorted result
    :param head_n: top %n
    :param gran: granularity
    """
    gran = 1000
    result = []
    for grp in rec_result.groupby('car'):
        temp = grp[1].values[:, -1].astype(float)
        idx = max(round(head_n / gran * len(temp)), 1)
        error = np.mean(temp[:idx])
        result.append([grp[0], int(error > threshold_n), error, threshold_n])
    return pd.DataFrame(result, columns=['car', 'predict', 'error', 'threshold_n'])

def evaluation(dataframe_std, dataframe):
    """
    calculated statistics
    :param dataframe_std:
    :param dataframe:
    :return:
    """

    # calculate auroc
#     print(dataframe) # error car
    _label = []
    for each_car in dataframe['car']:
        if int(each_car) in ind_car_num_list:
            _label.append(0)
        if int(each_car) in ood_car_num_list:
            _label.append(1)

    fpr, tpr, thresholds = metrics.roc_curve(_label, list(dataframe['error']), pos_label=1)
    auroc = auc(fpr, tpr)


    data = pd.merge(dataframe_std, dataframe, on='car')
    cm = confusion_matrix(data['label'].astype(int), data['predict'].astype(int))
    tn = cm[0, 0]
    fp = cm[0, 1]
    fn = cm[1, 0]
    tp = cm[1, 1]
    precision = tp / (tp + fp) if tp + fp != 0 else 0
    recall = tp / (tp + fn) if tp + fn != 0 else 0
    false_rate = fp / (tn + fp) if tn + fp != 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn) if tp + tn + fp + fn != 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0
    return f1, recall, false_rate, precision, accuracy, auroc

In [16]:
# ! ls ../AE_and_SVDD/traditional_save

In [ ]:
AUC_list = []

method = 'DeepSVDD'  # auto_encoder DeepSVDD
dataset = 'batterydatasetall_' # batterydatasetall_ batterydataset1_ batterydataset3_

for fold_num in range(0,5):


    # change to your corresponding dir
    y_train_pred = np.load(f'../AE_and_SVDD/traditional_save/{method}_normalize_{dataset}fold%d_y_train_pred.npy' % fold_num)
    y_train_scores = np.load(f'../AE_and_SVDD/traditional_save/{method}_normalize_{dataset}fold%d_y_train_scores.npy' % fold_num)
    y_test_pred = np.load(f'../AE_and_SVDD/traditional_save/{method}_normalize_{dataset}fold%d_y_test_pred.npy' % fold_num)
    y_test_scores = np.load(f'../AE_and_SVDD/traditional_save/{method}_normalize_{dataset}fold%d_y_test_scores.npy' % fold_num)
    y_test = np.load(f'../AE_and_SVDD/traditional_save/{method}_normalize_{dataset}fold%d_y_test.npy' % fold_num)

    print(y_train_pred.shape, y_train_scores.shape, y_test_pred.shape, y_test_scores.shape, y_test.shape)

    # replace the following two car number lists
    ind_ood_car_dict = np.load('../five_fold_utils/ind_odd_dict.npz.npy', allow_pickle=True).item()
#     ind_ood_car_dict = np.load('../five_fold_utils/ind_odd_dict1.npz.npy', allow_pickle=True).item()
#     ind_ood_car_dict = np.load('../five_fold_utils/ind_odd_dict3.npz.npy', allow_pickle=True).item()
    ind_car_num_list = ind_ood_car_dict['ind_sorted']
    ood_car_num_list = ind_ood_car_dict['ood_sorted']
    
    
    #
    car_number_train = ind_car_num_list[:int(fold_num * len(ind_car_num_list) / 5)] + ind_car_num_list[int((fold_num + 1) * len(ind_car_num_list) / 5):]
    car_number_test = ind_car_num_list[int(fold_num * len(ind_car_num_list) / 5):int((fold_num + 1) * len(ind_car_num_list) / 5)] + ood_car_num_list

    # build 3 column csv file
    all_car_dict = np.load('../five_fold_utils/all_car_dict.npz.npy', allow_pickle=True).item()

    car_train = []
    for each_num in car_number_train:
        for each_pkl in all_car_dict[each_num]:
            car_train.append(each_num)
    car_test = []
    for each_num in car_number_test:
        for each_pkl in all_car_dict[each_num]:
            car_test.append(each_num)
    print(len(car_train))
    print(len(car_test))
    car_train = np.array(car_train)
    car_test = np.array(car_test)

    print(y_train_scores.shape)
    print(y_test_scores.shape)

    car_label_train = []
    car_label_test = []

    for each in car_train:
        if each in ind_car_num_list:
            car_label_train.append(0)
        if each in ood_car_num_list:
            car_label_train.append(1)
    for each in car_test:
        if each in ind_car_num_list:
            car_label_test.append(0)
        if each in ood_car_num_list:
            car_label_test.append(1)
    car_label_train = np.array(car_label_train)
    car_label_test = np.array(car_label_test)

    data_train = np.vstack((car_label_train, car_train, y_train_scores)).T
    data_test = np.vstack((car_label_test, car_test, y_test_scores)).T

    data = np.vstack((data_train, data_test))

    print(data_train.shape)
    print(data_test.shape)
    print(data_train[:5, ])
    print(data.shape)

    validation_car_number = ind_car_num_list[:int(fold_num * len(ind_car_num_list) / 5)] + ind_car_num_list[int((fold_num + 1) * len(ind_car_num_list) / 5):] + ood_car_num_list[int(fold_num * len(ood_car_num_list) / 5):int((fold_num + 1) * len(ood_car_num_list) / 5)]
    test_car_number = ind_car_num_list[int(fold_num * len(ind_car_num_list) / 5):int((fold_num + 1) * len(ind_car_num_list) / 5)] + ood_car_num_list[:int(fold_num * len(ood_car_num_list) / 5)] + ood_car_num_list[int((fold_num + 1) * len(ood_car_num_list) / 5):]


    validation_data = []
    test_data = []

    for each_line in data:
        if each_line[1] in validation_car_number:
            validation_data.append(each_line)
        else:
            test_data.append(each_line)

    validation_data = np.array(validation_data)
    test_data = np.array(test_data)
    print(validation_data.shape, test_data.shape)

    rec_sorted_index = np.argsort(-validation_data[:, 2].astype(float))
    res = [validation_data[i][[1, 0, 2]] for i in rec_sorted_index]
    result = pd.DataFrame(res, columns=['car', 'label', 'rec_error'])
    result['car'] = result['car'].astype("int").astype("str")
    best_n, max_percent, granularity = find_best_percent(result, granularity_all=1000)
    head_n = best_n / granularity
    data_length = round(len(result) * head_n)
    threshold_n = result['rec_error'].values[data_length - 1].astype(float)
    print("threshold_n", threshold_n)
    print("trainset start,flag is", 'rec_error')
    #
    dataframe_std1 = pd.read_csv(os.path.join('../../../battery_dataset1/label', "label.csv"), dtype=object)
    dataframe_std2 = pd.read_csv(os.path.join('../../../battery_dataset2/label', "label.csv"), dtype=object)
    dataframe_std3 = pd.read_csv(os.path.join('../../../battery_dataset3/label', "label.csv"), dtype=object)
#     dataframe_std = pd.concat([dataframe_std1], axis=0,ignore_index=True)
#     dataframe_std = pd.concat([dataframe_std3], axis=0,ignore_index=True)
    dataframe_std = pd.concat([dataframe_std1, dataframe_std2, dataframe_std3], axis=0,ignore_index=True)
#     assert max(dataframe_std['car'].astype("int"))>300
#     assert 201 in dataframe_std['car'].astype("int")
    best_result, best_h, best_re, best_fa, best_f1, best_auroc = find_best_result(threshold_n, result, dataframe_std)
    # best_result.to_csv(os.path.join(self.args.result_path, "train_res.csv")),
    if dataframe_std.shape[0] == best_result.shape[0]:
        pass
    else:
        print('dataframe_std is ', dataframe_std.shape[0], '&&   dataframe is ', best_result.shape[0])
    print("best 1000 / %d:" % best_h)
    print("re:", best_re)
    print("fa:", best_fa)
    print("F1:", best_f1)
    rec_sorted_index = np.argsort(-test_data[:, 2].astype(float))
    res = [test_data[i][[1, 0, 2]] for i in rec_sorted_index]
    result = pd.DataFrame(res, columns=['car', 'label', 'rec_error'])
    result['car'] = result['car'].astype("int").astype("str")
    # result.to_csv(os.path.join(self.args.result_path, "test_segment_scores.csv"))
    test_result = charge_to_car(threshold_n, result, head_n=best_h)

    _score = list(test_result['error'])
    _label = []
    for each_car in test_result['car']:
        if int(each_car) in ind_car_num_list:
            _label.append(0)
        if int(each_car) in ood_car_num_list:
            _label.append(1)

    fpr, tpr, thresholds = metrics.roc_curve(_label, _score, pos_label=1)

    np.save(f'../AE_and_SVDD/traditional_save/robust_{method}_all_score_fold%d.npy' % fold_num, _score)
    np.save(f'../AE_and_SVDD/traditional_save/robust_{method}_labels_fold%d.npy' % fold_num, _label)

    # print(_label, _score)
    plt.plot(fpr, tpr)
    plt.show()
    AUC = auc(fpr, tpr)
    print(AUC)
    AUC_list.append(AUC)



In [ ]:
print(AUC_list)
print(np.mean(AUC_list))
print(np.std(AUC_list))